In [1]:
import pandas as pd
import numpy as np
from ark_tweet_nlp import CMUTweetTagger
from tqdm import tqdm_notebook as tqdm
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
import pickle
import copy
import re

/home/qrg-researchlab/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/qrg-researchlab/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/qrg-researchlab/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/qrg-researchlab/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [6]:
"""Loading the tagged data and the 5-words neighborhoods """
data = pickle.load(open('./label_tag_data.p', 'rb'))
actual_words_5 = pickle.load(open('./neighborhoods/actual_words_hashtag_free_5.p', 'rb'))
hashtag_words = pickle.load(open('./neighborhoods/hashtag_words.p','rb'))

In [7]:
"""Pattern flagging without hashtags"""

"""Structuring the words into stand alone identifiers, negative paired identifiers, negative indicators, BA indicators, and other fluff"""
stand_alone = ['boycott','bycott','boycot','bycot','avoid']
neg_paired = ['fly','use','travel','choose','book','recommend']
neg_ind = ['never','not','dont','shouldnt','wont','wouldnt']

"""Function to pick pattern"""
def check_pattern(pat):
    sa_ind = int(len(set(list(pat)).intersection(stand_alone))>0)
    np_ind = int( (len(set(list(pat)).intersection(neg_paired))>0) & (len(set(list(pat)).intersection(neg_ind))>0))
    never_again = int('never' in pat and 'again' in pat)
    return [sa_ind, np_ind, never_again]

ys = list(data.pi)

pred_2 = []
for k in tqdm(range(len(actual_words_5)), desc='Going through the neighborhoods for 2 conditions'):
    label = False
    for pat in actual_words_5[k]:
        pf = check_pattern(pat)
        """Common conditions labeling"""
        if (pf[0]==1 or pf[1]==1) and not label:
            pred_2.append(1)
            label = True
            break
    """Labeling as a non negative PI tweet if no condition is met"""
    if not label:
        pred_2.append(0)
        
pred_3 = []
for k in tqdm(range(len(actual_words_5)), desc='Going through the neighborhoods for 2 conditions'):
    label = False
    for pat in actual_words_5[k]:
        pf = check_pattern(pat)
        """Common conditions labeling"""
        if (pf[0]==1 or pf[1]==1 or pf[2]==1) and not label:
            pred_3.append(1)
            label = True
            break
    """Labeling as a non negative PI tweet if no condition is met"""
    if not label:
        pred_3.append(0)
        
        
print('Using 2 conditions : negative PI verb, negation word + positive PI verb without hashtags')
print('F1 score obtained through the fully supervised method : {}'.format(f1_score(y_true=ys, y_pred=pred_2)))
print('Precision obtained through the fully supervised method : {}'.format(precision_score(y_true=ys, y_pred=pred_2)))
print('Recall score obtained through the fully supervised method : {}'.format(recall_score(y_true=ys, y_pred=pred_2)))

print("\nUsing 3 conditions : negative PI verb, negation word + positive PI verb, presence of 'never again' phrase without hashtags")
print('F1 score obtained through the fully supervised method : {}'.format(f1_score(y_true=ys, y_pred=pred_3)))
print('Precision obtained through the fully supervised method : {}'.format(precision_score(y_true=ys, y_pred=pred_3)))
print('Recall score obtained through the fully supervised method : {}'.format(recall_score(y_true=ys, y_pred=pred_3)))


Using 2 conditions : negative PI verb, negation word + positive PI verb without hashtags
F1 score obtained through the fully supervised method : 0.608816705336427
Precision obtained through the fully supervised method : 0.6090993500464252
Recall score obtained through the fully supervised method : 0.608534322820037

Using 3 conditions : negative PI verb, negation word + positive PI verb, presence of 'never again' phrase without hashtags
F1 score obtained through the fully supervised method : 0.6710182767624021
Precision obtained through the fully supervised method : 0.6319672131147541
Recall score obtained through the fully supervised method : 0.7152133580705009


In [8]:
"""Pattern flagging with hashtags"""

"""Structuring the words into stand alone identifiers, negative paired identifiers, negative indicators, BA indicators, and other fluff"""
stand_alone = ['boycott','bycott','boycot','bycot','avoid']
neg_paired = ['fly','use','travel','choose','book','recommend']
neg_ind = ['never','not','dont','shouldnt','no','wont','wouldnt']

"""Function to pick pattern"""
def check_pattern(pat):
    sa_ind = int(len(set(list(pat)).intersection(stand_alone))>0)
    np_ind = int( (len(set(list(pat)).intersection(neg_paired))>0) & (len(set(list(pat)).intersection(neg_ind))>0))
    never_again = int('never' in pat and 'again' in pat)
    return [sa_ind, np_ind, never_again]

ys = list(data.pi)

pred_2 = []
for k in tqdm(range(len(actual_words_5)), desc='Going through the neighborhoods for 2 conditions'):
    label = False
    for pat in actual_words_5[k] + hashtag_words[k]:
        pf = check_pattern(pat)
        """Common conditions labeling"""
        if (pf[0]==1 or pf[1]==1) and not label:
            pred_2.append(1)
            label = True
            break
    """Labeling as a non negative PI tweet if no condition is met"""
    if not label:
        pred_2.append(0)
        
pred_3 = []
for k in tqdm(range(len(actual_words_5)), desc='Going through the neighborhoods for 2 conditions'):
    label = False
    for pat in actual_words_5[k]+ hashtag_words[k]:
        pf = check_pattern(pat)
        """Common conditions labeling"""
        if (pf[0]==1 or pf[1]==1 or pf[2]==1) and not label:
            pred_3.append(1)
            label = True
            break
    """Labeling as a non negative PI tweet if no condition is met"""
    if not label:
        pred_3.append(0)
        
        
print('Using 2 conditions : negative PI verb, negation word + positive PI verb with hashtags')
print('F1 score obtained through the fully supervised method : {}'.format(f1_score(y_true=ys, y_pred=pred_2)))
print('Precision obtained through the fully supervised method : {}'.format(precision_score(y_true=ys, y_pred=pred_2)))
print('Recall score obtained through the fully supervised method : {}'.format(recall_score(y_true=ys, y_pred=pred_2)))

print("\nUsing 3 conditions : negative PI verb, negation word + positive PI verb, presence of 'never again' phrase with hashtags")
print('F1 score obtained through the fully supervised method : {}'.format(f1_score(y_true=ys, y_pred=pred_3)))
print('Precision obtained through the fully supervised method : {}'.format(precision_score(y_true=ys, y_pred=pred_3)))
print('Recall score obtained through the fully supervised method : {}'.format(recall_score(y_true=ys, y_pred=pred_3)))


Using 2 conditions : negative PI verb, negation word + positive PI verb with hashtags
F1 score obtained through the fully supervised method : 0.6393146979260595
Precision obtained through the fully supervised method : 0.6219298245614036
Recall score obtained through the fully supervised method : 0.6576994434137291

Using 3 conditions : negative PI verb, negation word + positive PI verb, presence of 'never again' phrase with hashtags
F1 score obtained through the fully supervised method : 0.7538461538461539
Precision obtained through the fully supervised method : 0.6688218390804598
Recall score obtained through the fully supervised method : 0.8636363636363636
